# Paternity arrays

Tom Ellis, March 2017

Paternity arrays are the what sibship clustering is built on in FAPS. They contain information about the probability that each candidate male is the father of each individual offspring. This information is stored in a `paternityArray` object, along with other related information. A `paternityArray` can either be imported directly, or created from genotype data.

This notebook will examine how to:

1. Create a `paternityArray` from marker data.
2. Examine what information it contains.
3. Read and write a `paternityArray` to disk, or import a custom `paternityArray`.

Once you have made your `paternityArray`, the [next step]((https://github.com/ellisztamas/faps/blob/master/docs/04%20Sibship%20clustering.ipynb) is to cluster the individuals in your array into full sibship groups.

## Creating a `paternityArray` from genotype data

To create a `paternityArray` from genotype data we need to specficy `genotypeArray`s for the offspring, mothers and candidate males. Currently only biallelic SNP data are supported.

We will illustrate this with a small simulated example again with four adults and six offspring.

In [4]:
from faps import *
import numpy as np

np.random.seed(49) # this ensures you get exactly the same answers as I do.
allele_freqs = np.random.uniform(0.3,0.5,10)
mypop        = make_parents(4, allele_freqs, family_name='my_population')
progeny      = make_sibships(mypop, 0, [1,2], 3, 'myprogeny')

We need to supply a `genotypeArray` for the mothers. This needs to have an entry for for every offspring, i.e. six replicates of the mother.

In [5]:
mum_index = progeny.parent_index('mother', mypop.names) # positions in the mothers in the array of adults
mothers   = mypop.subset(mum_index) # genotypeArray of the mothers

To create the `paternityArray` we also need to supply information on the genotyping error rate (mu), and population allele frequencies. For the latter, in this case we can either take the population allele frequencies defined above, or estimate them from the data, which will give slightly different answers. The function `paternity_array` creates an object of class `paternityArray`.

In [6]:
mu = 0
sample_af = mypop.allele_freqs()
patlik = paternity_array(offspring = progeny, mothers = mothers, males= mypop, allele_freqs = sample_af, mu=mu)

## `paternityArray` structure

A `paternityArray` inherits information about individuals from found in a `genotypeArray`, for example:

In [7]:
print patlik.candidates
print patlik.mothers
print patlik.offspring

['my_population_0' 'my_population_1' 'my_population_2' 'my_population_3']
['my_population_0' 'my_population_0' 'my_population_0' 'my_population_0'
 'my_population_0' 'my_population_0']
['myprogeny_0' 'myprogeny_1' 'myprogeny_2' 'myprogeny_3' 'myprogeny_4'
 'myprogeny_5']


The most important part of the `paternityArray` is the likelihood array, which represent the log likelihood that each candidate male is the true father of each offspring individual. In this case it will be a 6x4 dimensional array, which we could view with:

In [8]:
patlik.lik_array

array([[ -90.99299961,  -39.88984047, -118.53440857,  -66.60457085],
       [ -90.99299961,  -39.88984047,  -64.88180425,  -39.88984047],
       [ -91.68614679,  -39.88984047, -119.22755575,  -67.29771803],
       [-117.93087355,  -40.58298765,  -40.58298765,  -41.27613483],
       [ -66.8277144 , -120.74738151,  -41.27613483,  -68.34754015],
       [ -64.74827286,  -92.982829  ,  -39.19669329,  -66.26809861]])

You can see that individuals one (the mother) and four are incompatible as fathers with any offspring, because the first and last columns all show log likelihoods of negative infinity. The second candidate is the only compatible father for the first full-sibship, and the third father is the only compatible father for the second sibship (second and third columns respectively).

In toy example we know the error rate to be zero, and have given it as such. In reality this will almost never be true, and we would usually need to incorporate an error rate. Here's what happens to `patlik` if we use a realistic error rate.

In [9]:
mu = 0.0015
patlik = paternity_array(offspring = progeny, mothers = mothers, males= mypop, allele_freqs = sample_af, mu=mu)
patlik.lik_array

array([[-48.72167478, -39.86958225, -55.12851883, -45.45139773],
       [-48.72167478, -39.86958225, -43.7414884 , -39.86958225],
       [-49.41182645, -39.87182465, -55.8186705 , -46.1415494 ],
       [-54.52688972, -40.55749152, -40.55600387, -41.24540079],
       [-45.67706652, -57.33461056, -41.25288275, -47.191492  ],
       [-43.60661151, -50.70285247, -39.18242774, -45.12103699]])

No candidate can now be excluded explicitly, but we can see that the true sires have log liklihoods around ten units higher than the imposters. This means that the true sire is around 22,000-fold more likely to be the true father than the other candidates. That's quite a lot!

The `paternityArray` also includes information that the true sire is not in the sample of candidate males. In this case this is not helpful, because we know sampling is complete, but in real examples is seldom the case. By default this is defined as the liklihood of generating the offspring genotypes given the known mothers genotype and alleles drawn from population allele frequencies. Here, values for the six offspring are higher than the likelihoods for the non-sires, indicating that they are no more likely to be the true sire than a random unrelated individual.

In [10]:
patlik.lik_absent

array([-43.98104781, -46.47441545, -45.06361603, -46.09532809,
       -47.20009302, -45.41073138])

This becomes more informative when we combine likelihoods about sampled and unsampled fathers. For fractional analyses we really want to know the probability that the father was unsampled vs sampled, and how probable it is that a single candidate is the true sire. To do this, we concatenate the values in `patlik.lik_absent` as an extra column to `patlik.likelihoods`, then normalise the array so that values in each row sum to one. It is straightforward to do this:

In [11]:
patprob = patlik.prob_array
print patprob.shape # dimensions of the prob_array

(6, 5)


Printing the shape of the array demonstrates that we have gained a column. If we sum the rows we see that they do indeed add up to one now. Probabilities are stored as log probabilities, so we have to exponentiate first.

In [12]:
np.exp(patprob).sum(axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.])

You can specify the proportion $\theta$ of the population of candidate males which are missing. The likelihoods for non-sampled parents will be weighted by $\theta$, and likelihoods for sampled candidates by $1-\theta$.

In [15]:
patprob2 = patlik.adjust_prob_array(missing_parents=0.1)

Often the mother is included in the sample of candidate males, either because you are using the same array for multiple families, or self-fertilisation is a biological possibility. In a lot of cases though the mother cannot simultaneously be the sperm/pollen donor, and it is necessary to set the rate of self-fertilisation to zero.

In [16]:
patlik.adjust_prob_array(selfing_rate=0)

array([[            -inf,  -1.99493854e-02,  -1.52788860e+01,
         -5.60176487e+00,  -4.13141495e+00],
       [            -inf,  -7.04172407e-01,  -4.57607856e+00,
         -7.04172407e-01,  -7.30900561e+00],
       [            -inf,  -7.42725188e-03,  -1.59542731e+01,
         -6.27715200e+00,  -5.19921863e+00],
       [            -inf,  -9.19505403e-01,  -9.18017758e-01,
         -1.60741467e+00,  -6.45734198e+00],
       [            -inf,  -1.60869630e+01,  -5.23518911e-03,
         -5.94384444e+00,  -5.95244546e+00],
       [            -inf,  -1.15250325e+01,  -4.60778144e-03,
         -5.94321704e+00,  -6.23291143e+00]])

The likelihoods for the mother have changed to 0 (negative infinity on the log scale). You can set any selfing rate between zero and one if you have a good idea of what the value should be and how much it varies. Otherwise it may be better to estimate the selfing rate from the data, or else estimate it some other way.

You can also set likelihoods for particular individuals to zero manually. You might want to do this if you wanted to test the effects of incomplete sampling on your results, or if you had a good reason to suspect that some candidates could not possibly be the sire (for example, if the data are multigenerational, and the candidate was born after the offspring). This is done with the argument `purge`.

In [18]:
patlik.adjust_prob_array(purge=0)

array([[            -inf,  -1.99493854e-02,  -1.52788860e+01,
         -5.60176487e+00,  -4.13141495e+00],
       [            -inf,  -7.04172407e-01,  -4.57607856e+00,
         -7.04172407e-01,  -7.30900561e+00],
       [            -inf,  -7.42725188e-03,  -1.59542731e+01,
         -6.27715200e+00,  -5.19921863e+00],
       [            -inf,  -9.19505403e-01,  -9.18017758e-01,
         -1.60741467e+00,  -6.45734198e+00],
       [            -inf,  -1.60869630e+01,  -5.23518911e-03,
         -5.94384444e+00,  -5.95244546e+00],
       [            -inf,  -1.15250325e+01,  -4.60778144e-03,
         -5.94321704e+00,  -6.23291143e+00]])

This has removed the first individual (notice that this is identical to the previous example, because in this case the first individual is the mother). If you supply an integer or a list of integers to `purge`, these will be treated as indexes of individuals to be removed. Alternatively you can supply a float between zero and one, which will be interpreted as a proportion of the candidates to be removed at random, which can be useful for simulations.

In [24]:
patprob3 = patlik.adjust_prob_array(purge=0.4)
np.isinf(patprob3).mean(1) # proportion missing along each row.

array([ 0.4,  0.4,  0.4,  0.4,  0.4,  0.4])

Of course, rows still need to sum to one. Luckily `adjust_prob_array` does that automatically.

In [29]:
np.exp(patprob3).sum(1)

array([ 1.,  1.,  1.,  1.,  1.,  1.])

A final important note is that `patlik.adjust_prob_array` always refers back to the original `patlik.lik_array` and `patlik.lik_absent`, which remain unchanged. Calling `adjust_prob_array` will not alter the data stored for `patlik.prob_array` unless you assign it yourself.

In [30]:
patlik.prob_array = patprob2

## Importing a `paternityArray`

Frequently you may wish to save an array and reload it. Otherwise, you may be working with a more exotic system than FAPS currently supports, such as microsatellite markers or a funky ploidy system. In this case you can create your own matrix of paternity likelihoods and import this directly as a `paternityArray`. Firstly, we can save the array we made before to disk by supplying a path to save to:

In [31]:
patlik.write('mypatlik.csv')

We can reimport it again using `read_paternity_array`. This function is similar to the function for importing a `genotypeArray`, and the data need to have a specific structure:

1. Offspring names should be given in the first column
2. Names of the mothers are usually given in the second column.
3. If known for some reason, names of fathers can be given as well.
4. Likelihood information should be given *to the right* of columns indicating individual or parental names, with candidates' names in the column headers.
5. The final column should specify a likelihood that the true sire of an individual has *not* been sampled. Usually this is given as the likelihood of drawing the paternal alleles from population allele frequencies.

In [32]:
patlik = read_paternity_array('mypatlik.csv', mothers_col=1, likelihood_col=2)

Of course, you can of course generate your own `paternityArray` and import it in the same way. This is especially useful if your study system has some specific marker type or genetic system not supported by FAPS.